In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, Permute,Conv1D, BatchNormalization, GlobalAveragePooling1D, concatenate, Input
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import Model
import pickle
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [32]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec  8 22:36:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000                On | 00000000:00:0B.0 Off |                  Off |
| 30%   31C    P8               19W / 230W|  23915MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [33]:
cd trajectory_10s

[Errno 2] No such file or directory: 'trajectory_10s'
/workspace/trajectory_10s


In [34]:
subjectList = pd.read_csv('participantID1.csv', header=None) # import the number participant IDs without the header
numSubjects = subjectList.count()[0]

subjectAlias = np.linspace(1,numSubjects,numSubjects).tolist()
outerSubjectAlias = subjectAlias
trialNames = ['NN','YN','NY','YY'] # NN = NNL + NNH, YN = YNL + YNH, NY = NYL + NHH, YY = YYL + YYH


In [35]:
with open('trajectoryDataFrame_win10_multilabel.pkl', 'rb') as file:
    # Call load method to deserialze
    trajectoryDataFrame = pickle.load(file)
with open('outputDataFrame_win10_multilabel.pkl', 'rb') as file:
    # Call load method to deserialze
    outputDataFrame = pickle.load(file)

In [36]:
epochLength = 350
parameterSpace = []
miniBatch = [256, 512]
featureAliases = [0, 1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed','thetaDot'], ['cumulativeSum','thetaDot']]
for m in miniBatch:
    for o in featureAliases:
        tempParamSpace = [m,o]
        parameterSpace.append(tempParamSpace)

In [37]:
parameterSpace

[[256, 0], [256, 1], [512, 0], [512, 1]]

In [38]:
def assignUserOutput(dataFrameData, requiredSubjects):
    requiredData = dataFrameData.loc[requiredSubjects].to_numpy()
    return requiredData

def assignUserTraj(dataFrameData, requiredSubjects, featureCombo, requiredFeatureCombo):
    requiredData = np.array([])
    for i in range(len(trajectoryDataFrame)):
        temp = dataFrameData[i].loc[requiredSubjects,featureCombo[requiredFeatureCombo]].to_numpy()
        temp = temp.reshape(temp.shape[0],1,temp.shape[1])
        if i == 0:
            requiredData = temp
        if i != 0 :
            requiredData = np.append(requiredData,temp, axis=1)
    return requiredData

def CNNlstmModel(numFeatures, seqLength, numCLasses, NUM_CELLS=8):

    #ip = Input(shape=(numFeatures, seqLength))
    ip = Input(shape=(seqLength, numFeatures))
    x = Permute((2, 1))(ip)
    x = LSTM(NUM_CELLS)(x)
    x = Dropout(0.8)(x)

    #y = Permute((2, 1))(ip)
    #y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(ip)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])
    #out = Dense(numCLasses, activation='softmax')(x)
    out = Dense(numCLasses, activation='sigmoid')(x)
    model = Model(ip, out)
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1)
    #model.compile(loss='categorical_crossentropy',optimizer =optimizer,metrics=['accuracy']) 
    model.compile(loss='BinaryCrossentropy',optimizer =optimizer,metrics=['accuracy']) 
    # model.summary()
    #model.summary()
    # add load model code here to fine-tune
    return model

In [39]:
print("Do you want to clear existing data? Type yes to clear and continue.")
a = input()

if a == "yes":
    print("Data Cleared.")
    i = 0
    j = 0
    k = 0
    outerParameterAccuracy = np.array([])
    outerParameterUsed = np.array([])
    innerParameterUsed = np.array([])
    innerParameterAccuracy = np.array([])

    innerParameterTrainAccuracy = np.array([])
    innerParameterTestAccuracy = np.array([])
    innerParameterUsedAggregate = np.array([])

    innerParameterTrainAccuracy = np.array([])
    innerParameterTestAccuracy = np.array([])
    innerParameterUsedAggregate = np.array([])

    innerAggregatedData = np.array([])
    outerAggregatedData = np.array([])
    with open('saveData/i_win10.pkl', 'wb') as file:
        pickle.dump(i, file)
    with open('saveData/j_win10.pkl', 'wb') as file:
        pickle.dump(j, file)
    with open('saveData/k_win10.pkl', 'wb') as file:
        pickle.dump(k, file)
    with open('saveData/outerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterAccuracy, file)
    with open('saveData/outerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterUsed, file)
    with open('saveData/innerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(innerParameterUsed, file)
    with open('saveData/innerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(innerParameterAccuracy, file)
    with open('saveData/innerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(innerAggregatedData, file)
    with open('saveData/outerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(outerAggregatedData, file)

Do you want to clear existing data? Type yes to clear and continue.


 yes


Data Cleared.


In [40]:
K1 = 3
K2 = 3
trainFraction = 0.8 # defines fraction of training data
testFraction = 1 - trainFraction

In [41]:
with open('saveData/i_win10.pkl', 'rb') as file:
    i = pickle.load(file)
with open('saveData/j_win10.pkl', 'rb') as file:
    j= pickle.load(file)
with open('saveData/k_win10.pkl', 'rb') as file:
    k = pickle.load(file)
with open('saveData/outerParameterAccuracy_win10.pkl', 'rb') as file:
    outerParameterAccuracy = pickle.load(file)
with open('saveData/outerParameterUsed_win10.pkl', 'rb') as file:
    outerParameterUsed = pickle.load(file)
with open('saveData/innerParameterUsed_win10.pkl', 'rb') as file:
    innerParameterUsed = pickle.load(file)
with open('saveData/innerParameterAccuracy_win10.pkl', 'rb') as file:
    innerParameterAccuracy = pickle.load(file)
with open('saveData/innerAggregatedData_win10.pkl', 'rb') as file:
    innerAggregatedData = pickle.load(file)
with open('saveData/outerAggregatedData_win10.pkl', 'rb') as file:
    outerAggregatedData = pickle.load(file)

print('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}'.format(i,j,k))
while i < K1:
    np.random.shuffle(outerSubjectAlias)
    numTrainDataOuter = int(trainFraction*len(outerSubjectAlias))
    numTestDataOuter = len(outerSubjectAlias) - numTrainDataOuter
    trainSubjectsOuterLoop = outerSubjectAlias[0:numTrainDataOuter]
    testSubjectsOuterLoop = outerSubjectAlias[len(trainSubjectsOuterLoop):len(outerSubjectAlias)]
    innerSubjectAlias = trainSubjectsOuterLoop


    while j < K2:
        print()
        print('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}'.format(i,j,k))
        np.random.shuffle(innerSubjectAlias)
        numTrainDataInner = int(trainFraction*len(innerSubjectAlias))
        numTestDataInner = len(innerSubjectAlias) - numTrainDataInner
        trainSubjectsInnerLoop = outerSubjectAlias[0:numTrainDataInner]
        testSubjectsInnerLoop = outerSubjectAlias[len(trainSubjectsInnerLoop): len(trainSubjectsInnerLoop) + numTestDataInner]
        #for k, kk in zip(parameterSpace, range(len(parameterSpace))):
        while k < len(parameterSpace):
            requiredFeatureCombo = parameterSpace[k][1]
            trainTrajData = assignUserTraj(trajectoryDataFrame,trainSubjectsInnerLoop, featureCombo, requiredFeatureCombo)
            trainOutputData = assignUserOutput(outputDataFrame,trainSubjectsInnerLoop)

            testTrajData = assignUserTraj(trajectoryDataFrame,testSubjectsInnerLoop, featureCombo, requiredFeatureCombo)
            testOutputData = assignUserOutput(outputDataFrame,testSubjectsInnerLoop)

            # dropoutRate, LSTM1, LSTM2, numDenseLayers, features, seq_length, numClasses
            features = trainTrajData.shape[2]
            seq_length = trainTrajData.shape[1]
            numClasses = trainOutputData.shape[1]
            mlModel = CNNlstmModel(features, seq_length, numClasses, NUM_CELLS=8)
            #mlModel = lstmModel(parameterSpace[k][0], parameterSpace[k][1], parameterSpace[k][2], parameterSpace[k][3], features, seq_length, numClasses)
            # print("Parameter Tested: " + str((kk+1)/len(parameterSpace)))
            print("\r Inner Loop Progress: {:03.2f} %".format((k+1)/len(parameterSpace)), end='')
            innerHistory = mlModel.fit(trainTrajData,trainOutputData,epochs=epochLength,batch_size=parameterSpace[k][0],validation_data=(testTrajData, testOutputData),verbose=0)
            innerParameterAccuracy = np.append(innerParameterAccuracy, innerHistory.history['val_accuracy'][-1])
            #print("\n Parameter Set: {:02.1f},  Test Accuracy: {:03.3f} %".format((k+1)/len(parameterSpace),innerParameterAccuracy[-1]), end='')
            if  innerParameterUsed.shape[0]==0:
                innerParameterUsed = np.array(parameterSpace[k]).reshape(1,len(parameterSpace[k]))
            else:
                innerParameterUsed = np.append(innerParameterUsed, np.array(parameterSpace[k]).reshape(1,len(parameterSpace[k])), axis=0)

            if  innerAggregatedData.shape[0]==0:
                temp = [i, j,k,parameterSpace[k][0],parameterSpace[k][1],
                                                innerHistory.history['val_accuracy'][-1],innerHistory.history['accuracy'][-1]]
                innerAggregatedData = np.array(temp).reshape(1,len(temp))
            else:
                temp = [i, j,k,parameterSpace[k][0],parameterSpace[k][1],
                                                innerHistory.history['val_accuracy'][-1],innerHistory.history['accuracy'][-1]]
                innerAggregatedData = np.append(innerAggregatedData, np.array(temp).reshape(1,len(temp)), axis=0)

            k = k + 1
            with open('saveData/innerParameterUsed_win10.pkl', 'wb') as file:
                pickle.dump(innerParameterUsed, file)
            with open('saveData/innerParameterAccuracy_win10.pkl', 'wb') as file:
                pickle.dump(innerParameterAccuracy, file)
            with open('saveData/k_win10.pkl', 'wb') as file:
                pickle.dump(k, file)
            with open('saveData/innerAggregatedData_win10.pkl', 'wb') as file:
                pickle.dump(innerAggregatedData, file)
            with open("progress.txt", "a+") as file:
                file.write('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}, EndAccuracy = {} , MaxAccuracy = {} \n'.format(i,j,k, innerHistory.history['val_accuracy'][-1], np.max(innerHistory.history['val_accuracy'])))
        j = j + 1
        k = 0
        with open('saveData/j_win10.pkl', 'wb') as file:
            pickle.dump(j, file)

    max_val = innerParameterAccuracy.max()
    maxAccuracyIndex = innerParameterAccuracy.argmax()
    optimalHyperparams = innerParameterUsed[maxAccuracyIndex] # Optimal Hyperparameters in the inner loop
    requiredFeatureCombo = int(optimalHyperparams[1])
    # Extract data for outer loop training
    trainTrajData = assignUserTraj(trajectoryDataFrame,trainSubjectsOuterLoop, featureCombo, requiredFeatureCombo)
    trainOutputData = assignUserOutput(outputDataFrame,trainSubjectsOuterLoop)

    testTrajData = assignUserTraj(trajectoryDataFrame,testSubjectsOuterLoop, featureCombo, requiredFeatureCombo)
    testOutputData = assignUserOutput(outputDataFrame,testSubjectsOuterLoop)

    # dropoutRate, LSTM1, LSTM2, numDenseLayers, features, seq_length, numClasses
    miniBatchSize = int(optimalHyperparams[0])
    requiredFeatureCombo = int(optimalHyperparams[1])
    features = trainTrajData.shape[2]
    seq_length = trainTrajData.shape[1]
    numClasses = trainOutputData.shape[1]

    # Train the data
    mlModel = CNNlstmModel(features, seq_length, numClasses, NUM_CELLS=8)
    #mlModel = lstmModel(dropoutRate, LSTM1_nodes, LSTM2_nodes, numDenseLayers, features, seq_length, numClasses)
    outerHistory = mlModel.fit(trainTrajData,trainOutputData,epochs=epochLength,batch_size=miniBatchSize,validation_data=(testTrajData, testOutputData),verbose=0)
    outerParameterAccuracy = np.append(outerParameterAccuracy, outerHistory.history['val_accuracy'][-1])

    if  outerParameterUsed.shape[0]==0:
        outerParameterUsed = np.array(optimalHyperparams).reshape(1,len(optimalHyperparams))
    else:
        outerParameterUsed = np.append(outerParameterUsed, optimalHyperparams.reshape(1,len(optimalHyperparams)), axis=0)
    if  outerAggregatedData.shape[0]==0:
                temp = [i,optimalHyperparams[0],optimalHyperparams[1],
                                                outerHistory.history['val_accuracy'][-1],outerHistory.history['accuracy'][-1]]
                outerAggregatedData = np.array(temp).reshape(1,len(temp))
    else:
        temp = [i,optimalHyperparams[0],optimalHyperparams[1],
                                                outerHistory.history['val_accuracy'][-1],outerHistory.history['accuracy'][-1]]
        outerAggregatedData = np.append(outerAggregatedData, np.array(temp).reshape(1,len(temp)), axis=0)

    # save state
    i = i + 1
    j = 0
    innerParameterAccuracy = np.array([])
    innerParameterUsed = np.array([])
    print(" ")
    print("==================Saving Data========================")
    with open('saveData/i_win10.pkl', 'wb') as file:
        pickle.dump(i, file)
    with open('saveData/outerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterAccuracy, file)
    with open('saveData/outerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterUsed, file)
    with open('saveData/j_win10.pkl', 'wb') as file:
        pickle.dump(j, file)
    with open('saveData/k_win10.pkl', 'wb') as file:
        pickle.dump(k, file)
    with open('saveData/outerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(outerAggregatedData, file)

Outer Loop no. = 0, Inner Loop no. = 0, Parameter Space no. = 0

Outer Loop no. = 0, Inner Loop no. = 0, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 0, Inner Loop no. = 1, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 0, Inner Loop no. = 2, Parameter Space no. = 0
 Inner Loop Progress: 1.00 % 
==================Saving Data========================

Outer Loop no. = 1, Inner Loop no. = 0, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 1, Inner Loop no. = 1, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 1, Inner Loop no. = 2, Parameter Space no. = 0
 Inner Loop Progress: 1.00 % 
==================Saving Data========================

Outer Loop no. = 2, Inner Loop no. = 0, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 2, Inner Loop no. = 1, Parameter Space no. = 0
 Inner Loop Progress: 1.00 %
Outer Loop no. = 2, Inner Loop no. = 2, Parameter Space no. = 0
 Inner Loop Pro